In [5]:
import gc
import pandas as pd
# 用户行为，使用format1进行加载
# 加载全量样本

# user_log = pd.read_csv('user_log_format1.csv', dtype={'time_stamp':'str'})
# user_info = pd.read_csv('user_info_format1.csv')
# train_data1 = pd.read_csv('train_format1.csv')
# submission = pd.read_csv('test_format1.csv')
# train_data = pd.read_csv('train_format2.csv')
# 加载小样本
user_log = pd.read_csv('sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('sample_user_info.csv')
train_data1 = pd.read_csv('train.csv')
submission = pd.read_csv('test.csv')
train_data = pd.read_csv('train_format2.csv')


In [ ]:
print(user_log.columns)
print(user_info.columns)
print(train_data1.columns)
print(submission.columns)
print(train_data.columns)

In [2]:
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
#print(matrix)

matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
del user_info, train_data1
gc.collect()
print(matrix)

        user_id  merchant_id label origin  age_range  gender
0         34176         3906   0.0  train          6       0
1         34176          121   0.0  train          6       0
2         34176         4356   1.0  train          6       0
3         34176         2217   0.0  train          6       0
4        230784         4818   0.0  train          0       0
...         ...          ...   ...    ...        ...     ...
522336   228479         3111   nan   test          6       0
522337    97919         2341   nan   test          8       1
522338    97919         3971   nan   test          8       1
522339    32639         3536   nan   test          0       0
522340    32639         3319   nan   test          0       0

[522341 rows x 6 columns]


In [3]:
# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
#print(matrix)

# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
#print(matrix)

# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
#print(matrix)

#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
print(matrix)


        user_id  merchant_id label origin    u1    u2  u3   u4   u5        u6  \
0         34176         3906   0.0  train   451   256  45  109  108  5.833333   
1         34176          121   0.0  train   451   256  45  109  108  5.833333   
2         34176         4356   1.0  train   451   256  45  109  108  5.833333   
3         34176         2217   0.0  train   451   256  45  109  108  5.833333   
4        230784         4818   0.0  train    54    31  17   20   19  5.166667   
...         ...          ...   ...    ...   ...   ...  ..  ...  ...       ...   
522336   228479         3111   nan   test  2004  1173  71  278  282  6.000000   
522337    97919         2341   nan   test    55    29  14   17   17  4.750000   
522338    97919         3971   nan   test    55    29  14   17   17  4.750000   
522339    32639         3536   nan   test    72    46  24   33   35  5.800000   
522340    32639         3319   nan   test    72    46  24   33   35  5.800000   

        ...  age_2  age_3  

In [4]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

44

In [5]:
# 使用机器学习工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.05)

In [14]:
# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=5,
    n_estimators=500,
    min_child_weight=100, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42,
    alpha = 1,
    learning_rate = 0.1
)
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10
)

[0]	validation_0-auc:0.628953	validation_1-auc:0.604987
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.635699	validation_1-auc:0.612151
[2]	validation_0-auc:0.640188	validation_1-auc:0.614992
[3]	validation_0-auc:0.647536	validation_1-auc:0.626393
[4]	validation_0-auc:0.65021	validation_1-auc:0.628468
[5]	validation_0-auc:0.650981	validation_1-auc:0.629216
[6]	validation_0-auc:0.651454	validation_1-auc:0.630103
[7]	validation_0-auc:0.651654	validation_1-auc:0.631064
[8]	validation_0-auc:0.651816	validation_1-auc:0.631611
[9]	validation_0-auc:0.652079	validation_1-auc:0.631973
[10]	validation_0-auc:0.653016	validation_1-auc:0.632914
[11]	validation_0-auc:0.654134	validation_1-auc:0.633404
[12]	validation_0-auc:0.655008	validation_1-auc:0.633897
[13]	validation_0-auc:0.655866	validation_1-auc:0.63575
[14]	validation_0-auc:0.655932	validation_1-auc:0.634802


[141]	validation_0-auc:0.71053	validation_1-auc:0.667268
[142]	validation_0-auc:0.710727	validation_1-auc:0.667329
[143]	validation_0-auc:0.710844	validation_1-auc:0.667441
[144]	validation_0-auc:0.711163	validation_1-auc:0.667341
[145]	validation_0-auc:0.711664	validation_1-auc:0.667965
[146]	validation_0-auc:0.711772	validation_1-auc:0.668102
[147]	validation_0-auc:0.711957	validation_1-auc:0.668276
[148]	validation_0-auc:0.712213	validation_1-auc:0.66811
[149]	validation_0-auc:0.712529	validation_1-auc:0.66825
[150]	validation_0-auc:0.71286	validation_1-auc:0.668624
[151]	validation_0-auc:0.713098	validation_1-auc:0.668358
[152]	validation_0-auc:0.713196	validation_1-auc:0.668449
[153]	validation_0-auc:0.713231	validation_1-auc:0.668526
[154]	validation_0-auc:0.713561	validation_1-auc:0.66856
[155]	validation_0-auc:0.713822	validation_1-auc:0.668578
[156]	validation_0-auc:0.713845	validation_1-auc:0.668397
[157]	validation_0-auc:0.714023	validation_1-auc:0.668518
[158]	validation_0-

XGBClassifier(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=100, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              silent=None, subsample=0.8, verbosity=1)

In [15]:
# model.fit(X_train, y_train)

prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction2.csv', index=False)